In [ ]:
#This notebook contains functions necessary to implement Shamir's secret sharing
#Note that this code and the code that will be implemented into the project uses a linear representation instead of a finite field. 
#For any enterprise development the code must be changed to include a finite field build around a significantly large prime such as 2^127 -1 

In [21]:
#This cell contains necessary imports
import numpy as np
from scipy.interpolate import lagrange 
import pandas as pd
from random import seed
from random import randint
from numpy.polynomial.polynomial import Polynomial
import Crypto
#bytes object initialization 
from Crypto.PublicKey import RSA
#import bcrypt
import rsa
import random
import math

In [64]:
def main():
    S_list = create_secret_value_table()
    #generates a list of secret indicies
    RSA_list = generate_RSA_keypairs()
    #generates a dataframe of public private keypairs
    S_df = pd.DataFrame({"Secret ID": S_list})
    S_RSA_DF = pd.concat([S_df, RSA_list], axis = 1)
    S_RSA_DF.to_csv('RSA_Keys.csv')
    
    
    #concatonates the two frames creating a pair between secret ID and each set of keys
        
main()

In [3]:
def block_processing(block):
    agent_list = we_need_agents()
    (S, fC1, fC2) = generate_function_coefficients()
    share_list = generate_Shamir_Shares(agent_list, S, fC1, fC2)
    encrypted_block = encrypt_the_block(block, S)
    return share_list, encrypted_block

In [30]:
#This cell is used to test the encryption and block processing. 
(share_list, encrypted_block) = block_processing(block_list[1])
#(share_list, encrypted_block) = block_processing('00000020ec80c6c8f61a7810eed8d1b08b0870f34a49d59d336e4a069b7cf6f6a50ef335166970c468a1ff44df44e7b0c2d20c5054cf430e0516b785f60993683b52ba893e6b235effff7f200000000002020000000001010000000000000000000000000000000000000000000000000000000000000000ffffffff0401660101ffffffff02d0fe052a01000000160014e6e0768d5239cc1446fbf32cd825adf05b5110560000000000000000266a24aa21a9ed32209fa7aa328b352828204d4b41d1a90177efdf4c3bce51b9ccbe0a78647de90120000000000000000000000000000000000000000000000000000000000000000000000000020000000001013ccb4a45689fe7df424c98f3decf9eaebdd73c753d41894853d798c6a76ffeff0000000017160014b1a7d3b0d1011a3475c32949d2786676f0a376b3feffffff02300410240100000016001458db024dd71efaf0184926a4c85be3023fa0080400e1f50500000000160014e6e0768d5239cc1446fbf32cd825adf05b5110560247304402200e72240746ec045668b8a2e30ecd36a761ecc9bab1b2cc039dbb7ba12c7218820220019d9e63738e1b6932b6bf331c3514dd85ae4c2221f19ef458184907fa7b3455012102098cfd632f00010e78b712508cae2fc48c47a033b9a3c14e30b0eb3a074a5e4800000000')
stringbs = '00000020ec80c6c8f61a7810eed8d1b08b0870f34a49d59d336e4a069b7cf6f6a50ef335166970c468a1ff44df44e7b0c2d20c5054cf430e0516b785f60993683b52ba893e6b235effff7f200000000002020000000001010000000000000000000000000000000000000000000000000000000000000000ffffffff0401660101ffffffff02d0fe052a01000000160014e6e0768d5239cc1446fbf32cd825adf05b5110560000000000000000266a24aa21a9ed32209fa7aa328b352828204d4b41d1a90177efdf4c3bce51b9ccbe0a78647de90120000000000000000000000000000000000000000000000000000000000000000000000000020000000001013ccb4a45689fe7df424c98f3decf9eaebdd73c753d41894853d798c6a76ffeff0000000017160014b1a7d3b0d1011a3475c32949d2786676f0a376b3feffffff02300410240100000016001458db024dd71efaf0184926a4c85be3023fa0080400e1f50500000000160014e6e0768d5239cc1446fbf32cd825adf05b5110560247304402200e72240746ec045668b8a2e30ecd36a761ecc9bab1b2cc039dbb7ba12c7218820220019d9e63738e1b6932b6bf331c3514dd85ae4c2221f19ef458184907fa7b3455012102098cfd632f00010e78b712508cae2fc48c47a033b9a3c14e30b0eb3a074a5e4800000000'
print(share_list)
print(encrypted_block)
print(len(stringbs))

[{8: 17916}, {1: 759}, {5: 7719}, {9: 22263}, {2: 1788}]
b'Cf\x8f%\xe1\xc6lgl\xc2\x92W\xd2e\xcd\xb9\x0c\x95\xf40\x9e\xcc+H,4\xde\xf5\x9bR[\x97\x06U\x8dEo\x8bCK._\x123\xe2\x95\xb3s\x95\xd0\xe6q\x01\x8dg\x8cq\xcc@\x18K\xec\x86\x86q\x96\x88\xea\xc7P6\x1d\x08\xa1\x03\xf5\xf3\xf2\x14\x94Q~\x9d\xfe\x81\x18\xae\x94\xbe\xe5\x0f[f\xb2\x89k"\xac\x12\xe7\x0f\xb4,\x15\xae\xf1\x16\xb8ac3\xdd\x03\x91\xeb\x90\x80Ej\xf6\x9d\x82\xcb6_m\x8c<\xa6O\xd7{\x16\xb8j\x92J!q>\xa6+\x08w\x80\xffdx\x19~\xd7\x03\xb5R\xf4\xc5\xd5\xae\xd0\t#\x8b\x9fC\xd8\x80\xaeG\n\x84q\xa7\x97\xef\x0f\xb8(\x97s_\x0bC\x8f\xeb\xd0F\xc5H\x83D\x03;\'\x1f\xb9!\x94+\x9c\xa9w\xcfH\x9c\xbb-\xb4h\x96\x1dQ`F\xe5\x9by\x80\xf9\xc7\x12\xa7\xc4\x03\x0f\x8d\x0b\x06\x9b\xf6\xbe\r\x9c\xb7M\xcb(\x93\x15\xc8\x90\xc7Z\xfa\xe4_i\xb3\x06D\x0eKb"\x96\x86\x1d\x82\xe20]\xb42\xeb\x85q\x8d\rO\xfd\xdd=-\xaf\x15\t~]\xb3\xd4\xd6\x93\x18\xc3\xdd)])\xca\'\x11\xedGm$k\xde\xcdE\xb7fk\xb1\xb3\xb1\xbb%\xa8>\xc4\x8f\xda\x03\x8fk\xa4Z\xf0?Y\x11\xe2F@\x99\x1byg\xed\x90H\

In [24]:
#This method split the hex of the bitcoin blocks and breaks them into strings and returns them as a list. Due to RSA4096's length limitations we are capped at 501 bytes, thus we must trim the blocks and receive them as a list. 
def break_the_block(block_text):
    length = len(block_text)
    block_list = []
    for i in range(math.ceil(length/500)):
        substring = block_text[(500*i):(500*(i+1))]
        block_list.append(substring)
    return block_list

In [27]:
block = '00000020f67472a866d2b526a22ab132c2ad46578d58c968a3ff4542bc31b89778b2f237032f0b5b9a90f0bf15bf4396f9d8733b55a8cef1e8dbcdbced04e2a3ddae2b33cd11295effff7f200100000009020000000001010000000000000000000000000000000000000000000000000000000000000000ffffffff0401680101ffffffff02c059062a0100000017a9148b7a1532abef9a6cfc79cbb89a0fb171c73cb5d7870000000000000000266a24aa21a9edcc9e9dd007027c2ee8606ab265410f0af99d08038ac1a2298577f1fd47769c790120000000000000000000000000000000000000000000000000000000000000000000000000020000000001010de9e45636dc1b3178cdf43311bb3dc31d88e14e400450648a663f2dc9e64088000000001716001446d96422bc6661867305deb392a8c5c07d18f4c4feffffff02080410240100000017a914fb5ca92fef360be3009d9a8348212d35c91bf65c8700e1f5050000000017a914502f9eb830f1ac210a79147721b8c04f881dbe71870247304402204c8c27a2d230bb5b87c4732c4f0c3a63cfe61a5e3cdcc27e4ba3fa58400b314e02206e72f5670fa0d698ef1306b791794ef4a5188dd6b0555b6a10dc25b759fbdf7901210314cec2b73d7865554856c0c2e429be5566d8565e1f5ad9c6412b3022cbe65d1a6700000002000000000101eccdcf3ade33f3a3cb08c052f113c40fab83ee0fc53174112d908bef6801c97200000000171600142fbf079e4720979ceda468fe952d234f44b9b5a6feffffff0240787d010000000017a914f7b47eceb061184194882bd226c3495c5d1fbd8487d07e92220100000017a91461c49f7032686aa30366fa726ffba51e76eb62f787024730440220239bb3edab14720a7999c726e87d7a20567ea8e8c619e013ec128689bd3bbf51022011634f861bccc0ae85aaa9d02173e8539d91669bcd23e23674ce19dc0123bb19012102e8679bf5d9d4a4212fa991a08c99138dadf9ea58341224f5ad824e9f0622c9416700000002000000000101421651589f2eee1066cbef91ccb40c8c974b6ca8b199513554ae9791dcc00f07000000001716001446d96422bc6661867305deb392a8c5c07d18f4c4feffffff02080410240100000017a91407c95ac481802bb9129abc7c3b6c4ccf7fdb18808700e1f5050000000017a914ff536bd86b1701410a8074bfa5da114dc1bcef5e8702473044022079aa0013b2d2ca870c7d3b52f1f17b77fb53e8c11b36cdce1a9b2802c89d448802204d000486d475ea3faaef06f8dc640617fbad1300be2d56032733bddfd153a68e01210314cec2b73d7865554856c0c2e429be5566d8565e1f5ad9c6412b3022cbe65d1a67000000020000000001015aafbfa67849221ee4fa3e0309589913d182f90f9b3e8b8e3c4e066958defefb010000001716001432287953b7fa55d096fa2ed8a8e9b0104ec08d11feffffff02183f7fe20000000017a914b10a7f4ddb41b282e8802c4b740c70728927d1dc8700e1f5050000000017a914c4abadf18e28efb488fb987f40ab8396257758aa870247304402202438b022f4cb03f014a613f70c5a4ccc89f0805fcb45d3ba9e458b32fb9e98ec02203c9e47d3e0a398d054ed536635cc7aa844c2c6b2597455361b79d442f88bed27012102a26fd1a7b45f8c5ca99c927df7e5acaf5994c62f79aa5f1ab5d55ae6bf2f4d0d670000000200000000010115a7808c4aa756b61a946545402053b697b647281736e1db4b70f7f9683feeb40000000017160014d46e7c4b226f46575ac96377b3e0adb990f5c656feffffff0290d9e31f0100000017a914545263be9c28f847ef7f485cbb3d33b7e5c4c59f87801d2c040000000017a914d3281d206f3fab6e5c5f94b28f622966f3caedc98702473044022003d2b2c2e15d4e3df9bcccb5992b1feffdad7f67b734b24118da317f0dd11f9b022012933ac8972309b549a9e4ab4078b186981bd7623967707d6673ab650a7da97f0121025cb0e64a4a6f6a5552c97527f847c2befa52b3a97cbf77ec2e3c652c9e51f33967000000020000000001010ca1e9cb8e888125e60c0bb84632bcda54bcea9bb51419e5d053f110995362df0100000017160014d8d9152f2f59bb080c57234bdd1e8da28dff550efeffffff029896751e0100000017a9141cf6695ef6f905b11112372dcbe01c786f78f2e68740db1c040000000017a9147a5f842f5c55e838cd5fce3f56f9451e89eeb84787024730440220630ea5630d6e5029a8b8c23a7d8a43090beab49e0a1b28b7487464c0a150ab8e02207c1a257c6fcadaa44c0cfa871c9829a8adb7ccf9c7d5531695afa145bfb02439012103932bfea3878d3648b73c6c0beb32ccb5c2187f200c35ab78fbf3416678fccf356700000002000000000101269198c3ffaa45b5841c7b74daa431d4fa37f73196d29b336f792893b357f37b0000000017160014260a2496f7c8b5022413b6e093ccf775e27120ccfeffffff02a04184df0000000017a9147d2e00d1257ca06d92776a59db863f322bc2348b8780f0fa020000000017a914629149372520a6628d6e283b930f3d451a7b41748702473044022076d92f685bf1075356cbc535d9816e8a5488b25adccf9f7889560dc9dd18f3a3022046d5b858a8a6b7a3551b1b64bd35d01b32a331e892f4fee9345c7544e772a633012103347b2450980be5de51522e26d38f7ed3c7da97a82f083bfe6fc1ce6cc697b99867000000020000000001017e8b8d8afbd2ffbc0fce8f1531ba95ccd98ed81898a8376c86839ec92dc58b2a00000000171600146d8357cfd20cd6deabfab19c5704220881beafa5feffffff0240787d010000000017a914904dabdd96203bcab0f5ab833052c33a24db7baf8768bc06de0000000017a914e3d5a402751bd735422ab1e9990d12355dbb3405870247304402205034b2d8e91c4ca64ec5f160185bc558adb01c1306ad511bfe996e21c4bbc2bc02205b65790d64854feeb57083c8289e6499edfde17e68ed3e8921f03fd3eb86f91601210253a348408365169532b7962e509d9ae7d4ac61d1aa9339ec4b2f594e335a278e67000000'
block_list = break_the_block(block)
len(block_list[1])

500

In [4]:
#This cell creates a random set of 20 secret values for S, values will range between 100 and 1000
def create_secret_value_table():
    seed(42)
    S_list = []
    for i in range(20):
        S_list.append(randint(100,1000))
    S_list.sort()
    return S_list
    

In [5]:
#This cell creates a table of 20 key pairs to be combined with the secret list to act as a reference. NOTE SCRIPT REQUIRES 4 CORE CPU
def generate_RSA_keypairs():
    cols = ['public key', 'private key']
    N = 20
    RSA_list = pd.DataFrame(columns = cols)
    for i in range(N):
        (pubkey, privkey) = rsa.newkeys(4096, poolsize = 6)
        RSA_list = RSA_list.append({'public key' : pubkey , 'private key' : privkey}, ignore_index = True)
    return RSA_list

In [ ]:
#This function creates a master key for the secrets table. Deciding against second round of encryption for this piece
#def generate_encrypted_keys():
#    (master_pub, master_priv) = rsa.newkeys(4096, poolsize 6)
#    S_RSA_slice = S_RSA_DF.copy()
#    S_RSA_slice.columns= ['SecretID', 'PubKey', 'PrivKey']
#    S_RSA_slice['EncryptedPrivate'] = S_RSA_slice.apply(lambda row: rsa.encrypt(row.PrivKey, master_pub), axis = 1)
#    S_RSA_slice
 
    

In [73]:
#S_df = pd.DataFrame({"Secret ID": S_list})
#S_RSA_DF = pd.concat([S_df, RSA_list], axis = 1)

In [6]:
#Function to select the agents that will be signing
def we_need_agents():
    agent_list = []
    for i in range(1,11):
        agent_list.append(i)
    agent_list = random.sample(agent_list, 5)
    return agent_list
    

In [11]:
#Function to load the secrets csv if it has already been created
def load_secrets_blocks():
    S_RSA_DF = pd.read_csv('RSA_Keys.csv')
    Block_DF = pd.read_csv('Blockdata.csv')
    return S_RSA_DF, Block_DF

In [13]:
(S_RSA_DF, Block_DF) = load_secrets_blocks()

In [15]:
Block_DF.iloc[0]['Raw_bytes']

'00000020f67472a866d2b526a22ab132c2ad46578d58c968a3ff4542bc31b89778b2f237032f0b5b9a90f0bf15bf4396f9d8733b55a8cef1e8dbcdbced04e2a3ddae2b33cd11295effff7f200100000009020000000001010000000000000000000000000000000000000000000000000000000000000000ffffffff0401680101ffffffff02c059062a0100000017a9148b7a1532abef9a6cfc79cbb89a0fb171c73cb5d7870000000000000000266a24aa21a9edcc9e9dd007027c2ee8606ab265410f0af99d08038ac1a2298577f1fd47769c790120000000000000000000000000000000000000000000000000000000000000000000000000020000000001010de9e45636dc1b3178cdf43311bb3dc31d88e14e400450648a663f2dc9e64088000000001716001446d96422bc6661867305deb392a8c5c07d18f4c4feffffff02080410240100000017a914fb5ca92fef360be3009d9a8348212d35c91bf65c8700e1f5050000000017a914502f9eb830f1ac210a79147721b8c04f881dbe71870247304402204c8c27a2d230bb5b87c4732c4f0c3a63cfe61a5e3cdcc27e4ba3fa58400b314e02206e72f5670fa0d698ef1306b791794ef4a5188dd6b0555b6a10dc25b759fbdf7901210314cec2b73d7865554856c0c2e429be5566d8565e1f5ad9c6412b3022cbe65d1a67000000020

In [8]:
#This cell generates two values fC1 and fC2 using random int for the generation of the 2nd degree polynomial 
def generate_function_coefficients():
    row = randint(0,19)
    S = S_RSA_DF.iloc[row]['Secret ID']
    fC1= randint(50,500)
    fC2= randint(50,500)
    return S, fC1, fC2

In [38]:
(S, fC1, fC2) = generate_function_coefficients()


792
90
75


In [9]:
#This cell generates the polynomial function for evaluation of shares, Shamir_Poly is a polynomial variable
#def generate_polynomial_function:
#    return Shamir_Poly

SyntaxError: invalid syntax (<ipython-input-9-428ae4495048>, line 2)

In [10]:
#This cell calculates the shares off the Shamir_Poly. This function should take in the five agents that signed the transaction, calculate the shares for all five, then return their relevant value.
def generate_Shamir_Shares (agent_list, S, fC1, fC2):
    shamir_shares = []
    for agent in agent_list:
        share = S + fC1*agent + fC2 * agent * agent
        shamir_shares.append({agent:share})
    return shamir_shares

In [128]:
index = S_RSA_DF.loc[S_RSA_DF['Secret ID'] == S].index[0]
string1 = S_RSA_DF.loc[S_RSA_DF['Secret ID'] == S]['private key'][index]
print(string1)

PrivateKey(59097076057511595382855397596573846106480327664542480084619224661019374318200613241196049679453660782721992319993478195130544310725705063919601186127631895520665933066472595209692914301289657501730191518100806554468658168971825589145491301467717680256605648565083091544360125786365009045289536364704848510093693700923589259122072182847941987929593980880071371695826751435815792853100241957946138173747228650727859010027801376198390814915809201260308165319043424958518552372624598132319052901559386276480433500655331305061239863274915164981752478879318773164392719656367086180941722303605317834857410309779411858335740210598596595404552828127026598543799143226677946044900253243728921619230487201066399093705509570614726275355543701375691319492639632179457321134240591763610137475955658376858456860546073595687225660187321531368505528826112728304326632757622631900666178153930045207692489621666308595000801578865699655145439043350679159135491289337302118708854177275780180796651550616056432

In [29]:
def encrypt_the_block(block, S):
    message = block.encode('utf-8')
    index = S_RSA_DF.loc[S_RSA_DF['Secret ID'] == S].index[0]
    string1 = S_RSA_DF.loc[S_RSA_DF['Secret ID'] == S]['private key'][index]
    list1 = string1.split(', ')
    list1[0] = list1[0][11:]
    list1[-1] = list1[-1][:-1]
    n = int(list1[0])
    e = int(list1[1])
    d = int(list1[2])
    p = int(list1[3])
    q = int(list1[4])
    priv_key = rsa.PrivateKey(n, e, d, p, q)
    encrypted_block = rsa.encrypt(message, priv_key)
    return encrypted_block

In [63]:
share_list

[{9: 23959}, {1: 1239}, {7: 15435}, {8: 19460}, {4: 6204}]

In [127]:
S = 792
index = S_RSA_DF.loc[S_RSA_DF['Secret ID'] == S].index[0]
string1 = S_RSA_DF.loc[S_RSA_DF['Secret ID'] == S]['private key'][index]
list1 = string1.split(', ')
list1[0] = list1[0][11:]
list1[-1] = list1[-1][:-1]
print(list1)

['59097076057511595382855397596573846106480327664542480084619224661019374318200613241196049679453660782721992319993478195130544310725705063919601186127631895520665933066472595209692914301289657501730191518100806554468658168971825589145491301467717680256605648565083091544360125786365009045289536364704848510093693700923589259122072182847941987929593980880071371695826751435815792853100241957946138173747228650727859010027801376198390814915809201260308165319043424958518552372624598132319052901559386276480433500655331305061239863274915164981752478879318773164392719656367086180941722303605317834857410309779411858335740210598596595404552828127026598543799143226677946044900253243728921619230487201066399093705509570614726275355543701375691319492639632179457321134240591763610137475955658376858456860546073595687225660187321531368505528826112728304326632757622631900666178153930045207692489621666308595000801578865699655145439043350679159135491289337302118708854177275780180796651550616056432653318831